In [1]:
from typing import List
from abc import ABC, abstractmethod
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()


class TradeRecord(Base):
    __tablename__ = 'TradeRecord'
    id = Column(Integer, primary_key=True)
    source_currency = Column(String)
    dest_currency = Column(String)
    amount = Column(Integer)
    price = Column(Float)

    def __init__(self, source, dest, amount, price):
        self.source_currency = source
        self.dest_currency = dest
        self.amount = amount
        self.price = price
        
    def __str__(self):
        return f'{self.source_currency}, {self.dest_currency}, {self.amount}, {self.price}'
    

In [4]:
class TradeProcessor:
    def __init__(self, filename):
        self.filename = filename

    def _log_message(self, message: str) -> None:
        print(message)

    def process_trades(self) -> None:
        raw_records = self._read_records()
        parsed_records = self._parse_records(raw_records)
        self._store_records(parsed_records)

    def _read_records(self) -> List[str]:
        trade_records: List[str] = []
        with open(self.filename) as data_source:
            for trade_record in data_source:
                trade_records.append(trade_record)
        return trade_records

    def _parse_records(self, trade_data: List[str]) -> List[TradeRecord]:
        trades = []
        for index, line in enumerate(trade_data):
            fields = line.split(',')
            if not self._validate_trade_data(fields, index + 1):
                continue
            trade = self._map_record_to_object(fields)
            trades.append(trade)
        return trades

    def _map_record_to_object(self, processed_record: List[str]) -> TradeRecord:
        in_curr = slice(0, 3);
        out_curr = slice(3, None)
        source_curr_code = processed_record[0][in_curr]
        dest_curr_code = processed_record[0][out_curr]
        trade_amount = int(processed_record[1])
        trade_price = float(processed_record[2])
        trade_record = TradeRecord(source_curr_code, dest_curr_code,trade_amount, trade_price)
        return trade_record

    def _store_records(self, trades: List[TradeRecord]) -> None:
        engine = create_engine('postgresql://postgres:u2402/501@localhost:5432/python')
        Session = sessionmaker(bind=engine)
        Base.metadata.create_all(engine)
        session = Session()
        for trade in trades:
            session.add(trade)
        session.commit()
        session.close()
        self._log_message(f'{len(trades)} records have saved')

    def _validate_trade_data(self, record: List[str], index: int) -> bool:
        if len(record) != 3:
            self._log_message(f'Line {index} malformed. Only {len(record)} field(s) found.')
            return False
        if len(record[0]) != 6:
            self._log_message(f'Trade currencies on line {index} malformed: {record[0]}')
            return False
        try:
            trade_amount = float(record[1])
        except ValueError:
            self._log_message(f"Trade amount on line {index} not a valid integer: '{record[1]}'")
            return False
        try:
            trade_price = float(record[2])
        except ValueError:
            self._log_message(f'Trade price on line {index} not a valid decimal:{record[2]}')
            return False
        return True

In [5]:
trade_processor = TradeProcessor('Data.txt')
trade_processor.process_trades()

Trade currencies on line 4 malformed: UIYUY
3 records have saved
